# Part 3 (Python)

# Resampling and Stacking to Data Cube

Import packages

In [1]:
import xarray as xr
import dask.array as da
import numpy as np
import os

Set Path

In [4]:
#os.chdir(r"your/path")

Open Crop Yield Data

In [11]:
spring_wheat = xr.open_dataset("data/wheat_stacked_rotated.nc", engine='netcdf4')
soybean = xr.open_dataset("data/soybean_stacked_rotated.nc", engine='netcdf4')
maize = xr.open_dataset("data/maize_stacked_rotated.nc", engine='netcdf4')

Rename Year

In [12]:
spring_wheat['z'] = list(range(1982,2017))
soybean['z'] = list(range(1982,2017))
maize['z'] = list(range(1982,2017))
spring_wheat = spring_wheat.sel(z=slice('1982', '2016'))
soybean = soybean.sel(z=slice('1982', '2016'))
maize = maize.sel(z=slice('1982', '2016'))
spring_wheat = spring_wheat.rename({'z': 'year'})
soybean = soybean.rename({'z': 'year'})
maize = maize.rename({'z': 'year'})

Open Climate Variables

In [6]:
seasonal = xr.open_dataset("data/seasonal.nc4", engine='netcdf4')

Resample (to Resolution of Crop Yield Data)

In [8]:
gdhy_lat = maize.latitude
gdhy_lon = maize.longitude

In [9]:
resampled_data = seasonal.interp(lat=gdhy_lat, lon=gdhy_lon, method='linear')

Expot as Datacube

In [24]:
maize_da = da.from_array(maize['variable'],chunks=(35,92,232))
soybean_da = da.from_array(soybean['variable'],chunks=(35,92,232))
wheat_da = da.from_array(spring_wheat['variable'],chunks=(35,92,232))

In [25]:
dims = ('year', 'latitude', 'longitude')
resampled_data['maize'] =(dims ,  maize_da)
resampled_data['soybean'] =(dims ,  soybean_da)
resampled_data['wheat'] =(dims ,  wheat_da)

In [26]:
resampled_data.to_netcdf('data/datacube_final.nc')